In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

# fmt:off
_ = [sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent / "python"] if str(path) not in sys.path] #fmt:on

In [ ]:
import litellm
from devtools import debug
from langchain_community.llms.edenai import EdenAI
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

from python.ai_core.llm import get_llm

llm = get_llm()


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


structured_llm = llm.with_structured_output(Joke)
debug(structured_llm.invoke("Tell me a joke about cats"))

In [ ]:
from devtools import debug
from langchain_community.chat_models.edenai import ChatEdenAI
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


llm_with_tools = llm.bind_tools([add])

query = "What is 11 + 11?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)

debug(ai_msg)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

In [ ]:
debug(ai_msg)

In [ ]:
from python.ai_core.llm import get_llm

llm_oai = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm_groq = ChatGroq(model="lLama3-70b-8192")
llm_eden = EdenAI(
    feature="text",
    provider="openai",
    model="gpt-3.5-turbo-instruct",
)

llm = get_llm(llm_id="llama3_8_deepinfra")


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

In [ ]:
llm_with_tools = llm.bind_tools([add])
llm_with_tools.invoke("What is 45 +  99 ?").tool_calls  # type: ignore # works

# llm_with_tools = llm_eden.bind_tools([add]).tool_calls   # AttributeError: 'EdenAI' object has no attribute 'bind_tools'
# llm_with_tools = llm_groq.bind_tools([add]).tool_calls   AttributeError: 'ChatGroq' object has no attribute 'tool_calls'

In [1]:
from langchain_core.pydantic_v1 import BaseModel, Field


class FunnyLevel(Enum):
    very_funny= 0
    not_so = 1

class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    funny_level : FunnyLevel = Field(description="Eval how the joke is funny")
    


structured_llm = llm_oai.with_structured_output(Joke)
debug(structured_llm.invoke("Tell me a joke about cats"))

NameError: name 'Enum' is not defined

In [ ]:
structured_llm = llm_eden.with_structured_output(Joke)

In [ ]:
from devtools import debug
from langchain_community.chat_models.edenai import ChatEdenAI
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


llm_with_tools = llm.bind_tools([add])

query = "What is 11 + 11?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Search Assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100"})
debug(r)

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

llm = EdenAI(
    feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
)

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"]),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)

In [ ]:
input_ = """i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""
result = agent_chain(input_)